## Delivery version 10 - Pseudo labeling - LSTM64-version

### Changed to batch 768 for faster

Model for creating pseudolabels: 2x5epoch 3x64LSTM

Main model: 3x5poch  2x128Bi-LSTM

public test set size: 56k, private teste set size: 376k.


This version has 3 lanes of embedding and also 2x and 3xLSTM64, in addition to old 2xLSTM128
Has focal loss.

Text preprocessing is done with lower() as it tested to give better accuracy than leaving capital letetrs.

### Quora Insincere Questions Classification

https://www.kaggle.com/c/quora-insincere-questions-classification

In [1]:
RUN_LOCAL = False # Set this false to run in Kaggle - removes printouts, local dev set prediction etc.
TRAIN_PERCENT = 0.9 #developing 0.9

batch_size = 768 # how many samples to process at once  - 768 around 10% faster in Kaggles K80 than 512

In [3]:
emb = 'glove'
n_epochs = 4 # how many times to iterate over all samples
max_features = 88000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 50 # max number of words in a question to use
embed_size = 300 # how big is each word vector
SEED = 1006
SEED = SEED+2
LOGFILE = 'logfile.txt'
folds = 2

In [4]:
import time
start = time.time()

import random
import pandas as pd
import numpy as np
import gc
import os
from sklearn.metrics import f1_score
import re
import torch
import time
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNLSTM
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm
tqdm.pandas(desc='Progress')

import string

# Display whole text of dataframe field and don't cut it
pd.set_option('display.max_colwidth', -1)    

/home/max/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
from keras.layers import multiply, concatenate 
from keras import backend as K
# import keras.activations
from keras.layers import Concatenate

# starting capital is a Layer, starting small is a function. Use layers.
from keras.layers import Multiply
from keras.layers import MaxPooling1D, Conv1D, SpatialDropout1D
from keras.layers.pooling import AveragePooling1D
from keras.layers.normalization import BatchNormalization

In [6]:
EMBEDDINGS_PATH = '../input/embeddings/'
EMBEDDING_FILE_GLOVE = f'{EMBEDDINGS_PATH}/glove.840B.300d/glove.840B.300d.txt'
EMBEDDING_FILE_FAST = f'{EMBEDDINGS_PATH}/wiki-news-300d-1M/wiki-news-300d-1M.vec'

EMBEDDING_FILE_PARA = f'{EMBEDDINGS_PATH}/paragram_300_sl999/paragram_300_sl999.txt'
EMBEDDING_FILE_WORD2VEC = f'{EMBEDDINGS_PATH}/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'

In [7]:
# REPEATABILITY
def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()
# kernel https://www.kaggle.com/hengzheng/pytorch-starter

# https://www.kaggle.com/ziliwang/baseline-pytorch-bilstm
def bestThresshold(train_y,train_preds):
    tmp = [0,0,0] # idx, cur, max
    delta = 0
    for tmp[0] in tqdm(np.arange(0.1, 0.501, 0.01)):
        tmp[1] = f1_score(train_y, np.array(train_preds)>tmp[0])
        if tmp[1] > tmp[2]:
            delta = tmp[0]
            tmp[2] = tmp[1]
    print('best threshold is {:.4f} with F1 score: {:.4f}'.format(delta, tmp[2]))
    return delta

def train_validate_test_split(df, df_y, train_percent=.6, validate_percent=.2, random_state=10):
    np.random.seed(random_state)
    perm = np.random.permutation(len(df))
    m = len(df)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df[perm[:train_end]]
    validate = df[perm[train_end:validate_end]]
    test = df[perm[validate_end:]]
    
    train_y = df_y[perm[:train_end]]
    validate_y = df_y[perm[train_end:validate_end]]
    test_y = df_y[perm[validate_end:]]

    return train, validate, test, train_y, validate_y, test_y, perm

def f1(y_true, y_pred):
    '''
    metric from here 
    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
    '''
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

import tensorflow as tf
# Keras version focal loss
# usage loss=[focal_loss(alpha=.25, gamma=2)]
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        pt_1 = K.clip(pt_1, 1e-3, .999)
        pt_0 = K.clip(pt_0, 1e-3, .999)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [8]:
# Faster embedding loading - but cutting some words possibly
# https://www.kaggle.com/syhens/speed-up-your-preprocessing
#max_words=200000

# def load_glove_slow(word_index, max_words=200000, embed_size=300)
# add if i >= max_words: continue 
# instead of 'if i >= max_features: continue' ? or next to it?

In [9]:
## FUNCTIONS FROM https://www.kaggle.com/gmhost/gru-capsule

def load_glove(word_index, max_words=1000000):
    EMBEDDING_FILE = f'{EMBEDDINGS_PATH}/glove.840B.300d/glove.840B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')[:300]
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))
    
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.005838499,0.48782197
    print(f'mean: {all_embs.mean()}, std: {all_embs.std()}')
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        #if i >= max_words: continue            
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix 
    
def load_fasttext(word_index):    
    EMBEDDING_FILE = f'{EMBEDDINGS_PATH}/wiki-news-300d-1M/wiki-news-300d-1M.vec'
    
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index../input/embeddings/
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector

    return embedding_matrix

def load_para(word_index):
    EMBEDDING_FILE = f'{EMBEDDINGS_PATH}/paragram_300_sl999/paragram_300_sl999.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.0053247833,0.49346462
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    #embedding_matrix = np.random.normal(emb_mean, 0, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    
    return embedding_matrix

In [10]:
def build_vocab(texts):
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab
#vocab = build_vocab(df['question_text'])

In [11]:
# https://www.kaggle.com/qqgeogor/keras-lstm-attention-glove840b-lb-0-043
from keras.engine.topology import Layer

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.supports_masking = True
        #self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        # eij = K.dot(x, self.W) TF backend doesn't support it

        # features_dim = self.W.shape[0]
        # step_dim = x._keras_shape[1]

        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
    #print weigthted_input.shape
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        #return input_shape[0], input_shape[-1]
        return input_shape[0],  self.features_dim

In [12]:
os.environ['OMP_NUM_THREADS'] = '4'

## Load data

In [13]:
df_train = pd.read_csv("../input/train.csv")
df_test = pd.read_csv("../input/test.csv")

In [14]:
# Randomize
np.random.seed(SEED)
trn_idx = np.random.permutation(len(df_train))
df_train = df_train.iloc[trn_idx]

df = pd.concat([df_train ,df_test],sort=True)
df_train.head(2)

,qid,question_text,target
338652,42591771fa3a278f03f9,Are you a giant animal smart monkey or human with look like giant monkey but its smart?,1
14898,02ed6292809db66abc6a,Do you think any person will create a better group than groups like the AKC?,0


In [15]:
with open(LOGFILE, "a") as myfile:
    myfile.write(f'Dataframes loaded. Time passed: {(time.time()-start)/60:.2f} min\n')

### Preprocess text

In [16]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x: # comparison makes faster
            x = x.replace(punct, f' {punct} ')
    return x

df_train["question_text"] = df_train["question_text"].progress_apply(lambda x: clean_text(x))
df_test["question_text"] = df_test["question_text"].apply(lambda x: clean_text(x))

HBox(children=(IntProgress(value=0, description='Progress', max=1306122, style=ProgressStyle(description_width='initial')), HTML(value='')))

In [17]:
def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

df_train["question_text"] = df_train["question_text"].progress_apply(lambda x: clean_numbers(x))
df_test["question_text"] = df_test["question_text"].apply(lambda x: clean_numbers(x))

HBox(children=(IntProgress(value=0, description='Progress', max=1306122, style=ProgressStyle(description_width='initial')), HTML(value='')))

In [18]:
specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}

def clean_special_chars(text):
    
    for s in specials:
        text = text.replace(s, specials[s])    
    return text

df_train["question_text"] = df_train["question_text"].progress_apply(lambda x: clean_special_chars(x))
df_test["question_text"] = df_test["question_text"].apply(lambda x: clean_special_chars(x))

HBox(children=(IntProgress(value=0, description='Progress', max=1306122, style=ProgressStyle(description_width='initial')), HTML(value='')))

In [19]:
df_train["question_text"] = df_train["question_text"].apply(lambda x: x.lower())
df_test["question_text"] = df_test["question_text"].apply(lambda x: x.lower())

In [20]:
# CONTRACTIONS
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }
mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}
additional = {'fortnite': 'game', 'pubg': 'game'}

# COMBINE BOTH FOR SINGLE RUN
new_dict = {**mispell_dict, **contraction_mapping, **additional}
mispell_dict = new_dict

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = _get_mispell(mispell_dict)

def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

df_train["question_text"] = df_train["question_text"].progress_apply(lambda x: replace_typical_misspell(x))
df_test["question_text"] = df_test["question_text"].apply(lambda x: replace_typical_misspell(x))

HBox(children=(IntProgress(value=0, description='Progress', max=1306122, style=ProgressStyle(description_width='initial')), HTML(value='')))

In [22]:
with open(LOGFILE, "a") as myfile:
    myfile.write(f'Text cleaned. Time passed: {(time.time()-start)/60:.2f} min\n')

#### Save preprocessed versions

In [23]:
if RUN_LOCAL:
    df_train.to_csv('train_preprocessed.csv')
    df_test.to_csv('test_preprocessed.csv')

#### Load

In [24]:
if RUN_LOCAL:
    df_train = pd.read_csv('train_preprocessed.csv')
    df_test =  pd.read_csv('test_preprocessed.csv')

### Tokenize

In [25]:
#vocab = build_vocab(df['question_text'])
#len(vocab)

In [26]:
# larger than one
# count = sum(1 for i in vocab.values() if i >= 2)
# print(count)
# 192036

In [27]:
def classify(val):
    res = []
    if val == 1:        res.append('is 1')    
    if val > 1:        res.append('> 1')
    if val > 2:        res.append('> 2')
    if val > 3:        res.append('> 3')
    if val > 4:        res.append('> 4')
    if val > 5:        res.append('> 5')
    if val > 6:        res.append('> 6')
    if val > 7:        res.append('> 7')
    if val > 8:        res.append('> 8')        
    return res

from collections import Counter

In [28]:
# Counter(classification for val in vocab.values() for classification in classify(val))

In [29]:
#print(vocab['crystal'])
#print(vocab['crystals'])

### Keras tokenize

In [30]:
list_sentences_train = df_train['question_text']
list_sentences_test = df_test['question_text']
list_sentences_combined = list_sentences_train.append(list_sentences_test, ignore_index=True)
tokenizer = Tokenizer(num_words=max_features, filters='\t\n')
tokenizer.fit_on_texts(list(list_sentences_combined))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [31]:
train_x = pad_sequences(list_tokenized_train, maxlen=maxlen)
test_x = pad_sequences(list_tokenized_test, maxlen=maxlen)
train_y = df_train['target'].values

### Save preprocessed

In [32]:
if RUN_LOCAL:
    np.save('train_x_preprocessed.npy', train_x)
    np.save('train_y_preprocessed.npy', train_y)
    np.save('test_x_preprocessed.npy', test_x)
# save tokenizer

### Load 

In [33]:
if RUN_LOCAL:
    train_x = np.load('train_x_preprocessed.npy')
    train_y = np.load('train_y_preprocessed.npy')
    test_x =  np.load('test_x_preprocessed.npy')

In [34]:
with open(LOGFILE, "a") as myfile:
    myfile.write(f'Tokenizing done. Time passed: {(time.time()-start)/60:.2f} min\n')

## Embedding

### General

In [35]:
# missing entries in the embedding are set using np.random.normal so we have to seed here too
word_index = tokenizer.word_index

### GLOVE

In [36]:
seed_everything()
#embedding_matrix = load_glove(word_index)
now = time.time()
embedding_matrix = load_glove(word_index, max_words=400000)
print(f'{(time.time() -now)/60:.2f} min\n')

# max_words=200k
# mean: -0.005838498938828707, std: 0.4878219664096832
# 3.54 min
# 3,77 min other run

mean: -0.005838498938828707, std: 0.4878219664096832
3.56 min



### difference is small between 200k and 400k!

### FASTTEXT

In [37]:
now = time.time()
seed_everything()
embedding_matrix_fast = load_fasttext(word_index)
print(f'{(time.time() -now)/60:.2f} min\n')

1.58 min



### Paragram

In [38]:
now = time.time()
seed_everything()
embedding_matrix_para = load_para(word_index)
print(f'{(time.time() -now)/60:.2f} min\n')

2.73 min



### Was alternative - getting mean of embeddings

### Was alternative - Concatenate embeddings

### Save embedding matrix + vocab

In [38]:
if RUN_LOCAL:
    np.save('embedding_matrix.npy', embedding_matrix)
    np.save('word_index.npy', word_index)

### Load embedding matrix + vocab

In [39]:
if RUN_LOCAL:
    embedding_matrix = np.load('embedding_matrix.npy')
    word_index = np.load('word_index.npy')

In [40]:
with open(LOGFILE, "a") as myfile:
    myfile.write(f'Embeddings ready. Time passed: {(time.time()-start)/60:.2f} min\n')

### MODEL

#### Basic example models

### MODELS

In [39]:
# ver 14
# 2x64LSTM
# 64 lstm     Double embedding lines
# 2x6 epoch for example

# 128 to 84 sec with LSTM 128 -> 64: about 1/3 of time away
# 84 -> 174 sec if emb trainable!!! slow. also as default seemed overfit

#2 rounds. Average Test-F1: 0.68723
#STD: 0.00101, low to high: 0.00202
#Time: 14.991060626506805 min


def create_model2x64():
    
    embedding_matrix1 = embedding_matrix # Glove
    embedding_matrix2 = embedding_matrix_fast    
    #embedding_matrix3 = embedding_matrix_para
    
    seed_everything()
    inp = Input(shape=(maxlen,))
    
    # EMBEDDING 1
    x = Embedding(max_features, embed_size, weights=[embedding_matrix1], trainable=False)(inp) # trainable=True
    x = Dropout(0.1)(x)
    
    #LSTM
    attention_input = CuDNNLSTM(64, return_sequences=True)(x)

    #LINE1: maxpool 
    max_pool = Dropout(0.2)(attention_input)
    max_pool = MaxPooling1D()(max_pool) # size 25 when concatenating
    #LINE2: avgpool
    avg_pool = Dropout(0.2)(attention_input)
    avg_pool = AveragePooling1D()(avg_pool)
    #LINE3: attention
    print(attention_input.shape) # input 256,
    # attn 64 works for both lstm 128 and 64?
    attention_probs = Dense(units=64, activation='softmax', name='attention_probs')(attention_input)  
    print(attention_probs.shape)
    attention_mul = Multiply()([attention_input, attention_probs])
    print(f'Attention_mul shape: {attention_mul.get_shape()}') # (?, 50, 256)
    print(max_pool) # (?, 25, 256)
    
    
    # EMBEDDING 2
    x2 = Embedding(max_features, embed_size, weights=[embedding_matrix2], trainable=False)(inp) # trainable=True!
    x2 = Dropout(0.1)(x2)
    
    #LSTM
    attention_input2 = CuDNNLSTM(64, return_sequences=True)(x2)
    #LINE1: maxpool 
    max_pool2 = Dropout(0.2)(attention_input2)
    max_pool2 = MaxPooling1D()(max_pool2) # size 25 when concatenating
    #LINE2: avgpool
    avg_pool2 = Dropout(0.2)(attention_input2)
    avg_pool2 = AveragePooling1D()(avg_pool2)
    #LINE3: attention
    attention_probs2 = Dense(units=64, activation='softmax', name='attention_probs2')(attention_input2)  
    attention_mul2 = Multiply()([attention_input2, attention_probs2])

  
    # CONCAT
    #conc = Concatenate(axis=1)([attention_mul, max_pool, avg_pool, conv1d])
    conc = Concatenate(axis=1)([attention_mul, max_pool, avg_pool, attention_mul2, max_pool2, avg_pool2])
    
    print(f'Concat shape: {conc.get_shape()}')
    
    #conc = Attention(512, 50)(conc)
    conc = Attention(200+0)(conc) # 50 from attention (2-way lstm?), 25 from maxpooling1D, 25 from avg_pool, 11 CNN maxpool
    
    # linear, relu, bnorm, dropout, out
    #conc = Dense(50, activation="relu")(conc)
    conc = Dense(100, activation="relu")(conc)  # no change in running time if 50 or 10
       
    #conc = BatchNormalization()(conc) #effec? cost on speed?
    conc = Dropout(0.1)(conc)
    x = Dense(1, activation="sigmoid")(conc)
    return Model(inputs=inp, outputs=x)

In [40]:
# ver13 Return sequences False.      Double embedding  
# 4 epoch?             

# on 512 batchsize 5 epoch 0.6915


def create_model2x128():
    embedding_matrix1 = embedding_matrix
    embedding_matrix2 = embedding_matrix_fast
    #embedding_matrix3 = embedding_matrix_para    
    
    seed_everything()
    inp = Input(shape=(maxlen,))
    # x = Embedding(max_features, embed_size, weights=[embedding_matrix1], trainable=True)(inp)
    x = Embedding(max_features, embed_size, weights=[embedding_matrix1], trainable=False)(inp) # trainable=True!
    x = Dropout(0.1)(x)
    
    #LSTM
    attention_input = (CuDNNLSTM(128, return_sequences=True))(x)
        
    #LINE1: maxpool 
    max_pool = Dropout(0.2)(attention_input)
    max_pool = MaxPooling1D()(max_pool) # size 25 when concatenating
    #LINE2: avgpool
    avg_pool = Dropout(0.2)(attention_input)
    avg_pool = AveragePooling1D()(avg_pool)
    
    #LINE3: attention
    print(attention_input.shape) # input 256,
    attention_probs = Dense(units=128, activation='softmax', name='attention_probs')(attention_input)  
    print(attention_probs.shape)
    attention_mul = Multiply()([attention_input, attention_probs])
    print(f'Attention_mul shape: {attention_mul.get_shape()}') # (?, 50, 256)
    print(max_pool) # (?, 25, 256)
    
    
    # EMBEDDING 2
    x2 = Embedding(max_features, embed_size, weights=[embedding_matrix2], trainable=False)(inp) # trainable=True!
    x2 = Dropout(0.1)(x2)
    
    #LSTM
    attention_input2 = CuDNNLSTM(128, return_sequences=True)(x2)
    #LINE1: maxpool 
    max_pool2 = Dropout(0.2)(attention_input2)
    max_pool2 = MaxPooling1D()(max_pool2) # size 25 when concatenating
    #LINE2: avgpool
    avg_pool2 = Dropout(0.2)(attention_input2)
    avg_pool2 = AveragePooling1D()(avg_pool2)
    #LINE3: attention
    attention_probs2 = Dense(units=128, activation='softmax', name='attention_probs2')(attention_input2)  
    attention_mul2 = Multiply()([attention_input2, attention_probs2])

    # CONCAT
    #conc = Concatenate(axis=1)([attention_mul, max_pool, avg_pool, conv1d])
    conc = Concatenate(axis=1)([attention_mul, max_pool, avg_pool, attention_mul2, max_pool2, avg_pool2])
    
    print(f'Concat shape: {conc.get_shape()}')
    
    #conc = Attention(512, 50)(conc)
    conc = Attention(200+0)(conc) # 50 from attention (2-way lstm?), 25 from maxpooling1D, 25 from avg_pool, 11 CNN maxpool
    
    # linear, relu, bnorm, dropout, out
#conc = Dense(50, activation="relu")(conc)
    conc = Dense(100, activation="relu")(conc)  # no change in running time if 50 or 10   
    
    #conc = Activation('relu')(conc)
    
    #conc = BatchNormalization()(conc) #effec? cost on speed?
    conc = Dropout(0.1)(conc)
    x = Dense(1, activation="sigmoid")(conc)
    return Model(inputs=inp, outputs=x)

In [41]:
# BI-directional VERSION

def create_model2x128_bi():
    embedding_matrix1 = embedding_matrix
    embedding_matrix2 = embedding_matrix_fast
    #embedding_matrix3 = embedding_matrix_para    
    
    seed_everything()
    inp = Input(shape=(maxlen,))
    # x = Embedding(max_features, embed_size, weights=[embedding_matrix1], trainable=True)(inp)
    x = Embedding(max_features, embed_size, weights=[embedding_matrix1], trainable=False)(inp) # trainable=True!
    x = Dropout(0.1)(x)
    
    #LSTM
    attention_input = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
    #attention_input = CuDNNLSTM(128, return_sequences=True, 
    #                            tf.keras.initializers.he_normal(seed=SEED))(x) # keyword follows...
        
    #LINE1: maxpool 
    max_pool = Dropout(0.2)(attention_input)
    max_pool = MaxPooling1D()(max_pool) # size 25 when concatenating
    #LINE2: avgpool
    avg_pool = Dropout(0.2)(attention_input)
    avg_pool = AveragePooling1D()(avg_pool)
    #LINE3: attention
    
    print(attention_input.shape) # input 256,

    attention_probs = Dense(units=256, activation='softmax', name='attention_probs')(attention_input)  
    print(attention_probs.shape)
    
    attention_mul = Multiply()([attention_input, attention_probs])
    print(f'Attention_mul shape: {attention_mul.get_shape()}') # (?, 50, 256)
    print(max_pool) # (?, 25, 256)
    
    
    # EMBEDDING 2
    x2 = Embedding(max_features, embed_size, weights=[embedding_matrix2], trainable=False)(inp) # trainable=True!
    x2 = Dropout(0.1)(x2)
    
    #LSTM
    attention_input2 = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x2)
    #LINE1: maxpool 
    max_pool2 = Dropout(0.2)(attention_input2)
    max_pool2 = MaxPooling1D()(max_pool2) # size 25 when concatenating
    #LINE2: avgpool
    avg_pool2 = Dropout(0.2)(attention_input2)
    avg_pool2 = AveragePooling1D()(avg_pool2)
    #LINE3: attention
    attention_probs2 = Dense(units=256, activation='softmax', name='attention_probs2')(attention_input2)  
    attention_mul2 = Multiply()([attention_input2, attention_probs2])

    # CONCAT
    #conc = Concatenate(axis=1)([attention_mul, max_pool, avg_pool, conv1d])
    conc = Concatenate(axis=1)([attention_mul, max_pool, avg_pool, attention_mul2, max_pool2, avg_pool2])
    
    print(f'Concat shape: {conc.get_shape()}')
    
    #conc = Attention(512, 50)(conc)
    conc = Attention(200+0)(conc) # 50 from attention (2-way lstm?), 25 from maxpooling1D, 25 from avg_pool, 11 CNN maxpool
    
    # linear, relu, bnorm, dropout, out
#conc = Dense(50, activation="relu")(conc)
    conc = Dense(100, activation="relu")(conc)  # no change in running time if 50 or 10   
    
    #conc = Activation('relu')(conc)
    
    #conc = BatchNormalization()(conc) #effec? cost on speed?
    conc = Dropout(0.1)(conc)
    x = Dense(1, activation="sigmoid")(conc)
    return Model(inputs=inp, outputs=x)

In [42]:
#15 - 3 embedding  3x64 LSTM           8 min epoch  0.687
# 64 lstm     Double embedding lines
# 4 epoch?

# 128 to 84 sec with LSTM 128 -> 64: about 1/3 of time away

#2 rounds. Average Test-F1: 0.68723
#STD: 0.00101, low to high: 0.00202
#Time: 14.991060626506805 min


def create_model3x64():
    embedding_matrix1 = embedding_matrix
    embedding_matrix2 = embedding_matrix_fast
    embedding_matrix3 = embedding_matrix_para
    
    seed_everything()
    inp = Input(shape=(maxlen,))
    # x = Embedding(max_features, embed_size, weights=[embedding_matrix1], trainable=True)(inp)
    x = Embedding(max_features, embed_size, weights=[embedding_matrix1], trainable=False)(inp) # trainable=True!
    x = Dropout(0.1)(x)
    
    
    # EMBEDDING 1
    #LSTM
    attention_input = (CuDNNLSTM(64, return_sequences=True))(x)
    #attention_input = CuDNNLSTM(128, return_sequences=True, 
    #                            tf.keras.initializers.he_normal(seed=SEED))(x) # keyword follows...        
    #LINE1: maxpool 
    max_pool = Dropout(0.2)(attention_input)
    max_pool = MaxPooling1D()(max_pool) # size 25 when concatenating
    #LINE2: avgpool
    avg_pool = Dropout(0.2)(attention_input)
    avg_pool = AveragePooling1D()(avg_pool)
    #LINE3: attention
    print(attention_input.shape) # input 256,
    # attn 64 works for both lstm 128 and 64?
    attention_probs = Dense(units=64, activation='softmax', name='attention_probs')(attention_input)  
    print(attention_probs.shape)
    attention_mul = Multiply()([attention_input, attention_probs])
    print(f'Attention_mul shape: {attention_mul.get_shape()}') # (?, 50, 256)
    print(max_pool) # (?, 25, 256)
    
    
    # EMBEDDING 2
    x2 = Embedding(max_features, embed_size, weights=[embedding_matrix2], trainable=False)(inp) # trainable=True!
    x2 = Dropout(0.1)(x2)
    #LSTM
    attention_input2 = CuDNNLSTM(64, return_sequences=True)(x2)
    #LINE1: maxpool 
    max_pool2 = Dropout(0.2)(attention_input2)
    max_pool2 = MaxPooling1D()(max_pool2) # 
    #LINE2: avgpool
    avg_pool2 = Dropout(0.2)(attention_input2)
    avg_pool2 = AveragePooling1D()(avg_pool2)
    #LINE3: attention
    attention_probs2 = Dense(units=64, activation='softmax', name='attention_probs2')(attention_input2)  
    attention_mul2 = Multiply()([attention_input2, attention_probs2])

    # EMBEDDING 3
    x3 = Embedding(max_features, embed_size, weights=[embedding_matrix3], trainable=False)(inp) # trainable=True!
    x3 = Dropout(0.1)(x3)
    #LSTM
    attention_input3 = CuDNNLSTM(64, return_sequences=True)(x3)
    #LINE1: maxpool 
    max_pool3 = Dropout(0.2)(attention_input3)
    max_pool3 = MaxPooling1D()(max_pool3) 
    #LINE2: avgpool
    avg_pool3 = Dropout(0.2)(attention_input3)
    avg_pool3 = AveragePooling1D()(avg_pool3)
    #LINE3: attention
    attention_probs3 = Dense(units=64, activation='softmax', name='attention_probs3')(attention_input3)  
    attention_mul3 = Multiply()([attention_input3, attention_probs3])
    

    #LINE4 CONVOLUTION
    #conv1d = Conv1D(256, 7, activation='relu')(x)
            #conv1d = Conv1D(256, 3, activation='relu')(conv1d)
    #conv1d = MaxPooling1D(pool_size=8)(conv1d)
            #conv1d = GlobalMaxPool1D()(conv1d)
            #conv1d = Dropout(0.2)(conv1d)
    #print(conv1d)
    
    # CONCAT
    #conc = Concatenate(axis=1)([attention_mul, max_pool, avg_pool, conv1d])
    #conc = Concatenate(axis=1)([attention_mul, max_pool, avg_pool, attention_mul2, max_pool2, avg_pool2])
    conc = Concatenate(axis=1)([attention_mul, max_pool, avg_pool, attention_mul2, max_pool2, avg_pool2, attention_mul3, max_pool3, avg_pool3])
    
    print(f'Concat shape: {conc.get_shape()}')
    
    #conc = Attention(512, 50)(conc)  
    conc = Attention(300+0)(conc) # 50 from attention (2-way lstm?), 25 from maxpooling1D, 25 from avg_pool, 11 CNN maxpool
                                    # 100 per emb line
    
    # linear, relu, bnorm, dropout, out
#conc = Dense(50, activation="relu")(conc)
    conc = Dense(100, activation="relu")(conc)  # no change in running time if 50 or 10.
    
    
    #conc = Activation('relu')(conc)
    
    #conc = BatchNormalization()(conc) #effec? cost on speed?
    conc = Dropout(0.1)(conc)
    x = Dense(1, activation="sigmoid")(conc)
    return Model(inputs=inp, outputs=x)

## Extra split for testing only
data are
train_x, train_y
test_x

In Final test_x is read from file. 
Here for testing we use part of labeled data to test_x

## Pseudo-labeling

2x64LSTM with glove+fast is 8 minute? 0.687 ?

### Train fast Pseudo-classifier

1) Train classifier using 100% training data.

2) Use it to predict the test-data - test_x

3) Pick highest confident predictions (close to 0 or close to 1) and assign the predicted label: as pseudo-label

4) add by combining to training-set


In [43]:
all_scores = []
all_train_preds = [] # original data
all_train_preds_plus = [] # for original + pseudo-labeled
all_final_preds = []

In [45]:
# About <8 minute, 0.68+  '2X64' LSTM 2-embedding model

#test
#n_epochs=1  
#folds = 2

n_epochs=7  
folds = 2
model_scores = []

training_start=time.time()
for i in range(folds):

    # Train / Val / Test -split
    seed_everything()
    X_tra, X_val, X_test, y_tra, y_val, y_test, permutation = train_validate_test_split(train_x, train_y, 
                                    train_percent=TRAIN_PERCENT, validate_percent=0, random_state=SEED+30+i+2)
    
    #model = Model(inputs=inp, outputs=x)
    model= create_model2x64()
    #model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy',f1])
   
    #model.compile(loss=[focal_loss(alpha=.25, gamma=2)], optimizer='rmsprop', metrics=['accuracy',f1])
    model.compile(loss=[focal_loss(alpha=.25, gamma=2)], optimizer='rmsprop', metrics=None)

    seed_everything()
    
    if RUN_LOCAL: # progress bar and predictions on dev set
        model.fit(X_tra, y_tra, batch_size=batch_size, epochs=n_epochs, validation_data=(X_test, y_test)); #verbose=2
        
    else:
        model.fit(X_tra, y_tra, batch_size=batch_size, epochs=n_epochs, validation_data=(X_val, y_val), verbose=2); #verbose=2

        # train_x vs X_tra  real train-set or this parts 0.9 of it
    train_preds = model.predict(train_x, batch_size=1024)
    delta = bestThresshold(train_y,train_preds)

    test_preds = model.predict(X_test, batch_size=1024)
    #print(f'Len test preds: {len(test_preds)}')    
    score = f1_score(y_test, np.array(test_preds)>delta)
    print(f'Test F1 {score:.5f} ({len(test_preds)} samples)')
    print()
    model_scores.append(score)    
    # PSEUDO
    all_train_preds.append( train_preds ) # for counting common F1-delta

    all_final_preds.append(  model.predict(test_x, batch_size=1024)  )
    
    
duration = time.time() - training_start
print(f'{len(model_scores)} rounds. Average Test-F1: {np.mean(model_scores):.5f}') 
print(f'STD: {np.std(model_scores):.5f}, low to high: {np.max(model_scores) - np.min(model_scores):.5f}')    
print(f'Time: {duration/60} min')

# all_scores.append([np.mean(model_scores), model_scores ])

(?, 50, 64)
(?, 50, 64)
Attention_mul shape: (?, 50, 64)
Tensor("max_pooling1d_3/Squeeze:0", shape=(?, 25, 64), dtype=float32)
Concat shape: (?, 200, 64)
Train on 1175509 samples, validate on 0 samples
Epoch 1/1
 - 75s - loss: 9.8236


HBox(children=(IntProgress(value=0, max=41), HTML(value='')))


best threshold is 0.3800 with F1 score: 0.6479
Test F1 0.63977 (130613 samples)

(?, 50, 64)
(?, 50, 64)
Attention_mul shape: (?, 50, 64)
Tensor("max_pooling1d_5/Squeeze:0", shape=(?, 25, 64), dtype=float32)
Concat shape: (?, 200, 64)
Train on 1175509 samples, validate on 0 samples
Epoch 1/1
 - 75s - loss: 9.8454


HBox(children=(IntProgress(value=0, max=41), HTML(value='')))


best threshold is 0.3500 with F1 score: 0.6470
Test F1 0.64886 (130613 samples)

2 rounds. Average Test-F1: 0.64431
STD: 0.00455, low to high: 0.00909
Time: 3.766754953066508 min


In [46]:
with open(LOGFILE, "a") as myfile:
    myfile.write(f'Pseudo-training done. Time passed: {(time.time()-start)/60:.2f} min\n')

### Combine Final-predictions to mean

In [49]:
new1 = np.hstack( all_final_preds)
new2 = pd.DataFrame(new1)
test_preds = np.mean(new2, axis=1)

In [50]:
#test_preds

### Create Pseudo-labels

In [51]:
df_pse = pd.DataFrame(test_preds)
df_pse.columns = ['preds']
#df_pse['y'] = y_test

In [52]:
df_pse['ones'] = df_pse['preds'].apply(lambda x: x>= 0.58)  #0.57
df_pse['zeros'] = df_pse['preds'].apply(lambda x: x<= 0.08)  # 0.09 ->  0.01 smaller
df_pse['ones'] =  (df_pse['ones'] == True).astype(int)
df_pse['zeros'] = (df_pse['zeros'] == True).astype(int)

In [53]:
np.sum(df_pse['ones']==1)

157

#### Build pseudo training set

In [54]:
# Indexes of matching rows
idx = df_pse.index[(df_pse['ones']==1 ) | (df_pse['zeros']==1)].tolist()

In [55]:
train_pseudo_x = X_test[idx]
train_pseudo_y = y_test[idx]

In [56]:
len(train_pseudo_x)

28599

#### Combine real train + pseudo 

In [57]:
# CONCAT real train + pseudo

train_plus_x = np.concatenate((train_x, train_pseudo_x), axis=0)
train_plus_y = np.concatenate((train_y, train_pseudo_y), axis=0)
 
# random permutation
rand = np.random.permutation(len(train_plus_y))
train_plus_x = train_plus_x[rand]
train_plus_y = train_plus_y[rand]


### Use Stratified K Fold to improve results

In [58]:
# if make 5 or 6 splits, can use only 2-4 of them also. Each fold leaves different 1/5 or 1/6 out?

from sklearn.model_selection import StratifiedKFold

n_splits=5

splits = list(StratifiedKFold(n_splits=n_splits, shuffle=False, random_state=SEED).split(train_plus_x, train_plus_y))
splits[:3]

[(array([ 266945,  266946,  266947, ..., 1334718, 1334719, 1334720]),
  array([     0,      1,      2, ..., 266942, 266943, 266944])),
 (array([      0,       1,       2, ..., 1334718, 1334719, 1334720]),
  array([266945, 266946, 266947, ..., 534039, 534040, 534041])),
 (array([      0,       1,       2, ..., 1334718, 1334719, 1334720]),
  array([531692, 531699, 531742, ..., 800915, 800916, 800917]))]

In [59]:
# splits[:1]

In [60]:
#for i, (train_idx, valid_idx) in enumerate(splits):    
# dig into the list with nested []
# train_idx, valid_idx = splits[:1][0][:]
# len(train_idx)
# len(valid_idx)
#train_plus_x[train_idx]

## Main training

In [61]:
# Main train:  2x128 LSTM,   about 16 min epoch, 0.69+, 20+ min on K80

# 3x64 runs 120s/epoch on 1070, 5x2x 2min = 20 min
# 3x64 on K80  201 s  

#test
#n_epochs=1 
#folds = 1

n_epochs=7 #6
folds = 3 

#    dur training 74 min, 3 folds x 5 epoch = 15 epoch
model_scores = []

training_start=time.time()

# DO NOT EXCEED NR of K-folds (or then code to go back fold 1)
assert folds <= n_splits

for i in range(folds): 
    
    # Check time remaingin from timelimit 2 hours - if too little remaining, break
    time_remain = 7200 - (time.time()-start)
    print(f'Remaining from 2 hours: {time_remain} s')
    
    
                        # 5 epochs * 2,5 min(k80 2x64) = 12,5 min/ 750 s +30% from pseudo_labels = 1000
                        # this is 3x64 so longer than 150s
    spare_time = 2000  # 7 epochs * 201 s = 1400 s  +30% from pseudo_labels = ... (faster batch now)
    if time_remain < spare_time:
         break
           
    
    # Train / Val / Test -split   TRAIN_PLUS version with pseudo_labeled!
#    seed_everything()
#    X_tra, X_val, X_test, y_tra, y_val, y_test, permutation = train_validate_test_split(train_plus_x, train_plus_y, 
#                                    train_percent=TRAIN_PERCENT, validate_percent=0, random_state=SEED)
 
#for i, (train_idx, valid_idx) in enumerate(splits):    

    # ASSIGN THE K-FOLD
    train_idx, valid_idx = splits[:1+i][0][:]
    X_tra = train_plus_x[train_idx]
    y_tra = train_plus_y[train_idx]
    X_test =train_plus_x[valid_idx]
    y_test =train_plus_y[valid_idx]   
    
    
    # random permutation
 #   rand = np.random.permutation(len(X_tra_plus))
#    X_tra = X_tra_plus[rand]
#    y_tra = y_tra_plus[rand]
    
    
    #model= create_model2x128_bi()  # create_model2x128()
    model= create_model3x64()
    
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', f1])
    #model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy',f1])
    #model.compile(loss=[focal_loss(alpha=.25, gamma=2)], optimizer='rmsprop', metrics=['accuracy',f1])
    model.compile(loss=[focal_loss(alpha=.25, gamma=2)], optimizer='rmsprop', metrics=None)
    
    #sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    #model.compile(loss='binary_crossentropy', optimizer=sgd)
    #https://faroit.github.io/keras-docs/1.2.2/optimizers/
    
    seed_everything()
    
    if RUN_LOCAL: # progress bar and predictions on dev set
        hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=n_epochs, validation_data=(X_test, y_test)); #verbose=2
        
    else:
        # FINAL ver
        #model.fit(X_tra, y_tra, batch_size=batch_size, epochs=n_epochs, validation_data=(X_val, y_val), verbose=2); #verbose=2
        model.fit(X_tra, y_tra, batch_size=batch_size, epochs=n_epochs, validation_data=None, verbose=2); #verbose=2
        
        # TESTING ver
        #model.fit(X_tra, y_tra, batch_size=batch_size, epochs=n_epochs, validation_data=(X_test, y_test), verbose=2); #verbose=2        
        
    # USE ORIGINAL TRAIN-SET     
    #train_preds = model.predict(X_tra, batch_size=1024)
    train_preds = model.predict(train_x, batch_size=1024)
    all_train_preds.append( train_preds ) # for counting common F1-delta
    
    delta = bestThresshold(train_y,train_preds)

    test_preds = model.predict(X_test, batch_size=1024)
    #print(f'Len test preds: {len(test_preds)}')    
    score = f1_score(y_test, np.array(test_preds)>delta)
    print(f'Test F1 {score:.5f} ({len(test_preds)} samples)')
    print()
    model_scores.append(score)      

    all_final_preds.append(  model.predict(test_x, batch_size=1024)  )
    
duration = time.time() - training_start
print(f'{len(model_scores)} rounds. Average Test-F1: {np.mean(model_scores):.5f}') 
print(f'STD: {np.std(model_scores):.5f}, low to high: {np.max(model_scores) - np.min(model_scores):.5f}')    
print(f'Time: {duration/60} min')

#all_scores.append([np.mean(model_scores), model_scores ])

Remaining from 2 hours: 6261.325834989548 s
(?, 50, 64)
(?, 50, 64)
Attention_mul shape: (?, 50, 64)
Tensor("max_pooling1d_7/Squeeze:0", shape=(?, 25, 64), dtype=float32)
Concat shape: (?, 300, 64)
Epoch 1/1
 - 100s - loss: 9.7434


HBox(children=(IntProgress(value=0, max=41), HTML(value='')))


best threshold is 0.2900 with F1 score: 0.6529
Test F1 0.65226 (266945 samples)

1 rounds. Average Test-F1: 0.65226
STD: 0.00000, low to high: 0.00000
Time: 2.5764235536257427 min


In [62]:
#del model
#gc.collect()
#model.summary()

In [63]:
# model.add(Dense(3, activation='softmax', activity_regularizer=activity_l2(0.0001)))

In [64]:
with open(LOGFILE, "a") as myfile:
    myfile.write(f'Finished main training at: {(time.time()-start)/60:.2f} min\n')

In [65]:
# K-fold

In [66]:
# Test batch 512 instead of 1024

# 512 runs slower per epoch, but seems converge faster. reaches conv at 5 epochs instead of 7 
# and is over 0.6915 !!

# 512 - slower: 140s vs 128s/1024
#BUT overfits, cut from 7 to 5 epochs?

# 1024 was 
#3 rounds. Average Test-F1: 0.69069
#STD: 0.00268, low to high: 0.00623
#Time: 47.08462586005529 min

In [67]:
with open(LOGFILE, "a") as myfile:
    myfile.write(f'Training done. Time passed: {(time.time()-start)/60:.2f} min\n')
    
with open(LOGFILE, "a") as myfile:
    myfile.write(f'Duration of training: {duration/60:.2f} min\n')    

### Combine predictions from K-fold models

In [68]:
# Combine version - mean
new1 = np.hstack( all_train_preds)
new2 = pd.DataFrame(new1)
train_preds = np.mean(new2, axis=1)

In [69]:
# For test set predictions   
new1 = np.hstack( all_final_preds)
new2 = pd.DataFrame(new1)
final_preds = np.mean(new2, axis=1)    

### Combined delta

In [70]:
delta = bestThresshold(train_y,train_preds)
#score = f1_score(y_test, np.array(test_preds)>delta)
#print(score)

HBox(children=(IntProgress(value=0, max=41), HTML(value='')))


best threshold is 0.3300 with F1 score: 0.6538


In [71]:
# if RUN_LOCAL == False:
submission = df_test[['qid']].copy()
submission['prediction'] = (final_preds > delta).astype(int)
submission.to_csv('submission.csv', index=False)
!head submission.csv

qid,prediction
00014894849d00ba98a9,0
000156468431f09b3cae,0
000227734433360e1aae,0
0005e06fbe3045bd2a92,0
00068a0f7f41f50fc399,0
000a2d30e3ffd70c070d,0
000b67672ec9622ff761,0
000b7fb1146d712c1105,0
000d665a8ddc426a1907,0


In [72]:
with open(LOGFILE, "a") as myfile:
    myfile.write(f'End of code reached. Total time: {(time.time()-start)/60:.2f} min\n')